<a href="https://colab.research.google.com/github/kanchanraiii/SecureRag/blob/master/Faiss_gemini_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
!pip install -qU langchain-google-genai scikit-learn spacy
!python -m spacy download en_core_web_sm

import os
import getpass
import re
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

print("✅ All libraries are installed and the API key is set.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 125.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Enter API key for Google Gemini: ··········
✅ All libraries are installed and the API key is set.


In [ ]:
!pip install faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 69.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving finance_dataset.jsonl to finance_dataset.jsonl


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
file_path = "finance_dataset.jsonl"
chunks = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        document_text = f.read()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
    )
    chunks = text_splitter.split_text(document_text)

    print(f"✅ Successfully loaded and chunked '{file_path}'. Found {len(chunks)} chunks.")
    if chunks:
        print("\n--- Sample Chunk 1 ---")
        print(chunks[0])

except FileNotFoundError:
    print(f"❌ Error: The file '{file_path}' was not found.")
    print("Please make sure you have uploaded the file and the name is correct.")

✅ Successfully loaded and chunked 'finance_dataset.jsonl'. Found 9045 chunks.

--- Sample Chunk 1 ---
{"name": "Tejas BhatQagar", "account_number": "HYZH9588211631790", "balance": 869258.08, "loan_application": {"amount": 375634.85, "status": "approved", "application_date": "2023-06-25"}, "transaction_history": [{"date": "2025-01-06", "amount": 49243.76, "type": "debit", "description": "orchestrate strategic models"}, {"date": "2025-07-01", "amount": 49243.76, "type": "credit", "description": "architect compelling infrastructures"}, {"date": "2025-05-12", "amount": 49243.76, "type": "debit", "description": "integrate rich eyeballs"}], "email": "yashasvi62@example.net", "phone": "09577266631", "pan": "BMZMR13953"}


In [ ]:
class RAGPipeline:
    def __init__(self, embedding_model_name: str, llm_name: str):
        print("--- Initializing Models and Filters ---")
        self.embedding_model = GoogleGenerativeAIEmbeddings(model=embedding_model_name)
        self.llm = ChatGoogleGenerativeAI(model=llm_name)
        self.vector_store = {}

        self.nlp = spacy.load("en_core_web_sm")
        self.REGEX_PATTERNS = {
            "EMAIL": r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",
            "PHONE": r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}",
            "CREDIT_CARD": r"\b(?:\d[ -]*?){13,16}\b",
            "SSN": r"\b\d{3}-\d{2}-\d{4}\b",
        }
        print("--- Models and Filters Initialized ---")

    def _input_filter(self, query: str, threshold: float = 0.5) -> (bool, str):
        """Filters the user's query for sensitive information."""
        doc = self.nlp(query)
        pii_count = sum(1 for ent in doc.ents if ent.label_ in ["PERSON", "GPE", "LOC", "ORG"])
        pii_count += sum(len(re.findall(pattern, query)) for pattern in self.REGEX_PATTERNS.values())

        if len(query.split()) > 0 and pii_count / len(query.split()) > threshold:
            return True, "Query blocked due to high concentration of sensitive information."
        return False, "Query is safe."

    def _output_filter(self, response: str) -> str:
        """Redacts sensitive information from the RAG model's output."""
        doc = self.nlp(response)
        redacted_text = list(response)
        for ent in doc.ents:
            if ent.label_ in ["PERSON", "GPE", "LOC", "ORG"]:
                start, end = ent.start_char, ent.end_char
                redacted_text[start:end] = f"[{ent.label_}]"
        redacted_text = "".join(redacted_text)
        for pii_type, pattern in self.REGEX_PATTERNS.items():
            redacted_text = re.sub(pattern, f"[{pii_type}]", redacted_text)
        return redacted_text

    def build_vector_store(self, text_chunks: list, batch_size: int = 100):
        """Builds the vector store by embedding chunks in smaller, trackable batches."""
        total_chunks = len(text_chunks)
        print(f"\n--- Building Vector Store for {total_chunks} chunks ---")
        for i in range(0, total_chunks, batch_size):
            batch_chunks = text_chunks[i:i + batch_size]
            batch_embeddings = self.embedding_model.embed_documents(batch_chunks)
            for j, (chunk, embedding) in enumerate(zip(batch_chunks, batch_embeddings)):
                self.vector_store[i + j] = {"text": chunk, "embedding": np.array(embedding).reshape(1, -1)}
            print(f"Processed {min(i + batch_size, total_chunks)} / {total_chunks} chunks...")
        print("--- Vector Store Created ---\n")

    def _get_single_embedding(self, text: str) -> np.ndarray:
        embedding = self.embedding_model.embed_query(text)
        return np.array(embedding).reshape(1, -1)

    def run_query(self, query: str):
        print(f"Processing query: '{query}'")
        print("="*30)

        is_sensitive, message = self._input_filter(query)
        if is_sensitive:
            print(f"Input Filter Action: {message}")
            return

        query_embedding = self._get_single_embedding(query)

        similarities = []
        for i, data in self.vector_store.items():
            sim = cosine_similarity(query_embedding, data["embedding"])[0][0]
            similarities.append((sim, data["text"]))
        similarities.sort(key=lambda x: x[0], reverse=True)
        context = [text for sim, text in similarities[:2]]

        print("\n--- Retrieved Context ---")
        for c in context:
            print(f"- {c[:150]}...")
        print("-" * 25, "\n")

        context_str = "\n".join(context)
        prompt = f"Context:\n{context_str}\n\nQuery: {query}\n\nAnswer:"
        result = self.llm.invoke(prompt)

        print(f"\nOriginal Generated Response: '{result.content}'")
        safe_response = self._output_filter(result.content)
        print(f"Final (Redacted) Response: '{safe_response}'\n")

print("✅ RAG Pipeline class is defined with integrated filters.")

✅ RAG Pipeline class is defined with integrated filters.


In [ ]:
if 'chunks' in locals() and chunks:
    pipeline = RAGPipeline(
        embedding_model_name="models/text-embedding-004",
        llm_name="gemini-1.5-flash"
    )

    pipeline.build_vector_store(chunks)

    test_query = "Who is Teerth Badal?"

    # --- THIS IS THE CORRECTED LINE ---
    pipeline.run_query(test_query)

else:
    print("❌ Cannot run the pipeline because no chunks were loaded.")

--- Initializing Models and Filters ---
--- Models and Filters Initialized ---

--- Building Vector Store for 9045 chunks ---
Processed 100 / 9045 chunks...
Processed 200 / 9045 chunks...
Processed 300 / 9045 chunks...
Processed 400 / 9045 chunks...
Processed 500 / 9045 chunks...
Processed 600 / 9045 chunks...
Processed 700 / 9045 chunks...
Processed 800 / 9045 chunks...
Processed 900 / 9045 chunks...
Processed 1000 / 9045 chunks...
Processed 1100 / 9045 chunks...
Processed 1200 / 9045 chunks...
Processed 1300 / 9045 chunks...
Processed 1400 / 9045 chunks...
Processed 1500 / 9045 chunks...
Processed 1600 / 9045 chunks...
Processed 1700 / 9045 chunks...
Processed 1800 / 9045 chunks...
Processed 1900 / 9045 chunks...
Processed 2000 / 9045 chunks...
Processed 2100 / 9045 chunks...
Processed 2200 / 9045 chunks...
Processed 2300 / 9045 chunks...
Processed 2400 / 9045 chunks...
Processed 2500 / 9045 chunks...
Processed 2600 / 9045 chunks...
Processed 2700 / 9045 chunks...
Processed 2800 / 90